In [1]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit -*-
# -*- date : 2025-04-24 -*-
# -*- Last revision: 2025-04-09 by roduit -*-
# -*- python version : 3.11.11 -*-
# -*- Description: Notebook that summarizeses the main results-*-

# <center> EE-452: Network Machine Learning </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Graph-based EEG Analysis </center>
---

In [2]:
#import libraries
import pandas as pd
from pathlib import Path
import sys

#import modules
import constants
from transform_func import *
from cnn_base import CnnBase
from dataloader import load_data
from utils import *

%load_ext autoreload
%autoreload 2

In [10]:
parent_dir = Path.cwd().resolve().parent.parent
os.chdir(parent_dir)

# Load Data

## Train

In [12]:
cfg = read_yml('project/config/exp/cnn/basic_cnn.yml')


In [13]:
loader_train = load_data(cfg.get("datasets")[0])

## Validation

In [15]:
loader_val = load_data(cfg.get("datasets")[1])

## Test

In [16]:
loader_test = load_data(cfg.get("datasets")[2])

In [18]:
for test_images, test_labels in loader_test:  
    sample_image = test_images[0]    # Reshape them according to your needs.
    sample_label = test_labels[0]

In [20]:
sample_label

tensor(0)

# Models

## Basic CNN

In [10]:
basic_cnn = CnnModel(input_shape=19, output_shape=1)

In [11]:
basic_cnn.train_model(loader_tr=loader_train, num_epochs=1)

Training: 100%|██████████| 1/1 [00:02<00:00,  2.77s/it]


In [14]:
basic_cnn.create_submission(loader_test, 'test.csv')

Submission file created at test.csv


,id,label
0,tensor(0),0
1,tensor(0),0
2,tensor(0),0
3,tensor(0),0
4,tensor(0),0
...,...,...
3609,tensor(0),0
3610,tensor(0),0
3611,tensor(0),0
3612,tensor(0),0
